In [ ]:
from redashAPI import RedashAPIClient

In [ ]:
with open('connect_to_redash', 'r') as f:
    text = f.read().split('\n')
    API_KEY = text[0]
    REDASH_HOST = text[1]

In [ ]:
Redash = RedashAPIClient(API_KEY, REDASH_HOST)

In [ ]:
res = Redash.get('data_sources')

# JOIN

# Необходимо получить списки наименований товаров в заказах
Описание запроса:

1) Делаем INNER JOIN подзапроса и таблицы products (с именами продуктов).
- Подзапрос: колонка product_ids в таблице orders - это массив, поэтому необходимо вытащить id продуктов при помощи UNNEST

2) далее имена полученные имена возвращаем в массив при помощи array_agg

In [ ]:
Redash.create_query(res.json()[0]['id'], 
                    "Match ids with names", 
                    "SELECT \
                        order_id, \
                        array_agg(name) AS product_names \
                    FROM (SELECT  \
                        order_id, \
                        UNNEST(product_ids) AS prod \
                    FROM orders) AS A \
                    JOIN products AS B \
                        ON A.prod = B.product_id \
                    GROUP BY order_id \
                    LIMIT 1000;")

# Выясните, какие пары товаров покупают вместе чаще всего.
Описание запроса:

Сделаем таблицу в CTE, чтобы дважды не писать одно и то же, так как в этом запросе необходимо сделать SELF JOIN.

CTE: 
- необходимо отсортировать и оставить только не отмененные заказы, поэтому в WHERE подзапросе найдем отмененные заказы и отсортируем при помощи NOT IN
- колонка product_ids в таблице orders - это массив, поэтому необходимо вытащить id продуктов при помощи UNNEST
- соединим с таблицей products, чтобы соотнести имена и id продуктов

Произведем SELF JOIN по id заказов и отфильтруем совпадающие id продуктов, так как они не могут составить пару. Далее делаем пары и группируем по ним. Считаем их количество.

In [ ]:
Redash.create_query(res.json()[0]['id'], 
                    "Pairs are bought often", 
                    "WITH complete_orders AS (SELECT \
                                                order_id, \
                                                name \
                                            FROM \
                                                (SELECT order_id, UNNEST(product_ids) AS goods \
                                                FROM orders \
                                                WHERE order_id NOT IN (SELECT order_id FROM user_actions \
                                                                       WHERE action = 'cancel_order')) AS A \
                                            JOIN products AS B \
                                            ON A.goods = B.product_id) \


                    SELECT \
                        array_sort(A.name || ARRAY[B.name]) AS pair \
                        COUNT(DISTINCT A.order_id) as count_pair \
                    FROM complete_orders AS A \
                    JOIN complete_orders AS B \
                        ON A.order_id = B.order_id AND A.name != B.name \
                    GROUP BY array_sort(A.name || ARRAY[B.name]) \
                    ORDER BY count_pair DESC, pair;")

# Оконные функции

# Посчитать, сколько заказов сделал и сколько отменил каждый пользователь на момент совершения нового действия
Иными словами, для каждого пользователя в каждый момент времени посчитать две накопительные суммы — числа оформленных и числа отменённых заказов. Если пользователь оформляет заказ, то число оформленных им заказов увеличивайте на 1, если отменяет — увеличивается на 1 количество отмен. В результате получатся три новые колонки с динамическими показателями, которые изменяются во времени с каждым новым действием пользователя.

Используем таблицу user_actions, где записаны действия пользователей. В SELECT отбираем необходимые колонки. Считаем, используя оконные функции и фильтруя через FILTER, количество оформленных и отмененных заказов в столбцах created_orders и canceled_orders соответственно. Далее считаем в колонке cancel_rate долю отмененных заказов на каждый момент времени

In [ ]:
Redash.create_query(res.json()[0]['id'], 
                    "Create and cancel orders by users", 
                    "SELECT \
                        user_id, \
                        order_id, \
                        action, \
                        time, \
                        COUNT(order_id) FILTER(WHERE action = 'create_order') \
                            OVER(PARTITION BY user_id ORDER BY time) AS created_orders, \
                        COUNT(order_id) FILTER(WHERE action = 'cancel_order') \
                            OVER(PARTITION BY user_id ORDER BY time) AS canceled_orders, \
                        ROUND(1.0 * COUNT(order_id) FILTER(WHERE action = 'cancel_order') \
                            OVER(PARTITION BY user_id ORDER BY time) \
                          / COUNT(order_id) FILTER(WHERE action = 'create_order') \
                              OVER(PARTITION BY user_id ORDER BY time), 2) AS cancel_rate \
                    FROM user_actions \
                    ORDER BY user_id, order_id, action, time \
                    LIMIT 1000;")

# Рассчить ежедневную выручку сервиса и прирост (относительный и абсолютный)
Таблица в подзапросе:
- необходимо отсортировать и оставить только не отмененные заказы, поэтому в WHERE подзапросе найдем отмененные заказы и отсортируем при помощи NOT IN
- колонка product_ids в таблице orders - это массив, поэтому необходимо вытащить id продуктов при помощи UNNEST

Соединим подзапрос с таблицей products, чтобы соотнести имена и id продуктов

Группируем по дням, чтобы считать ежедневную выручку. Считаем сумму. И при помощи оконных функций и функций смещения считаем ежедневный прирост выручки. Чтобы пропусков в первой строке не было, используем COALESCE, чтобы заменить на 0

In [ ]:
Redash.create_query(res.json()[0]['id'], 
                    "Service daily revenue", 
                    "SELECT \
                        creation_time::DATE AS date, \
                        ROUND(SUM(price), 1) AS daily_revenue, \
                        COALESCE(NULL, \
                                 ROUND(SUM(price) \
                                 - LAG(SUM(price)) OVER(ORDER BY creation_time::DATE), 1), 0) \
                            AS revenue_growth_abs, \
                        COALESCE(NULL, \
                                 ROUND(SUM(price) \
                                 / LAG(SUM(price)) OVER(ORDER BY creation_time::DATE) * 100 - 100, 1), 0) \
                            AS revenue_growth_percentage \
                    FROM \
                    (SELECT \
                        order_id, \
                        creation_time, \
                        UNNEST(product_ids) AS product_id \
                    FROM orders \
                    WHERE order_id NOT IN (SELECT \
                                           order_id \
                                           FROM user_actions \
                                           WHERE action = 'cancel_order')) AS A \
                    JOIN products AS B \
                        ON A.product_id = B.product_id \
                    GROUP BY creation_time::DATE \
                    ORDER BY date;")